#### This notebook explains the Abstract Syntax Tree (AST). 

It parses a simple method to show the structure of its nodes, then implements a basic RuboCop-style cop to demonstrate how it works.

This tutorial walks you through the process:

- [Static vs. Dynamic Code Analysis: How RuboCop Reads Ruby’s Mind](https://medium.com/jungletronics/static-vs-dynamic-code-analysis-how-rubocop-reads-rubys-mind-c3e190a28420)

- Explore the key differences between static and dynamic analysis — and discover how RuboCop inspects your code’s Abstract Syntax Tree (AST) to catch issues before your program even runs.

Enjoy!



In [1]:
`rails -v`

"Rails 8.0.4\n"

In [2]:
`ruby -v`

"ruby 3.4.2 (2025-02-15 revision d2930f8e7a) +PRISM [x86_64-linux]\n"

_note_: Backticks (`) in Ruby run a shell command and return its output as a string.

### Method Under Test

In [3]:
def 
  greet(name) 
  puts "Hello, #{name}" 
end
greet('Gilberto')

Hello, Gilberto


### AST

In [4]:
require 'ripper'
require 'pp'

code = <<~RUBY
  def greet(name)
    puts "Hello, \#{name}"
  end
RUBY

"def greet(name)\n  puts \"Hello, \#{name}\"\nend\n"

In [5]:
pp Ripper.sexp(code)

[:program,
 [[:def,
   [:@ident, "greet", [1, 4]],
   [:paren,
    [:params, [[:@ident, "name", [1, 10]]], nil, nil, nil, nil, nil, nil]],
   [:bodystmt,
    [[:command,
      [:@ident, "puts", [2, 2]],
      [:args_add_block,
       [[:string_literal,
         [:string_content,
          [:@tstring_content, "Hello, ", [2, 8]],
          [:string_embexpr, [[:var_ref, [:@ident, "name", [2, 17]]]]]]]],
       false]]],
    nil,
    nil,
    nil]]]]


[:program, [[:def, [:@ident, "greet", [1, 4]], [:paren, [:params, [[:@ident, "name", [1, 10]]], nil, nil, nil, nil, nil, nil]], [:bodystmt, [[:command, [:@ident, "puts", [2, 2]], [:args_add_block, [[:string_literal, [:string_content, [:@tstring_content, "Hello, ", [2, 8]], [:string_embexpr, [[:var_ref, [:@ident, "name", [2, 17]]]]]]]], false]]], nil, nil, nil]]]]

## Let's Practice!

Here is a mini-cop to print offenses if found, or “No offenses” if everything is fine.

In [6]:
require "parser/ruby34"  # loads parser for Ruby 3.4.x

def check_double_quotes(code)
  parser = Parser::CurrentRuby.new
  buffer = Parser::Source::Buffer.new('(example)')
  buffer.source = code
  ast = parser.parse(buffer)

  offenses = []

  check_node = lambda do |node|
    return unless node.is_a?(Parser::AST::Node)

    node.children.each { |child| check_node.call(child) if child.is_a?(Parser::AST::Node) }

    if node.type == :str
      source = node.location.expression.source
      if source.start_with?('"') && !source.include?('#{')
        offenses << "⚠️  Offense at line #{node.location.line}: Prefer single quotes for #{source}"
      end
    end
  end

  check_node.call(ast)

  if offenses.empty?
    puts "✅ No offenses detected!"
  else
    puts offenses
  end
end


:check_double_quotes

#### RUNNING

In [7]:
old_verbose, $VERBOSE = $VERBOSE, nil
require 'parser/current'
$VERBOSE = old_verbose


code1 = <<~RUBY
  name = "Gilberto"
  puts "Hello, \#{name}"
  puts "Static analysis is cool"
RUBY

"name = \"Gilberto\"\nputs \"Hello, \#{name}\"\nputs \"Static analysis is cool\"\n"

In [8]:
check_double_quotes(code1)

⚠️  Offense at line 1: Prefer single quotes for "Gilberto"
⚠️  Offense at line 3: Prefer single quotes for "Static analysis is cool"


In [9]:
code2 = <<~RUBY
  name = 'Gilberto'
  puts "Hello, \#{name}"
  puts 'Static analysis is cool'
RUBY

"name = 'Gilberto'\nputs \"Hello, \#{name}\"\nputs 'Static analysis is cool'\n"

In [10]:
check_double_quotes(code2)

✅ No offenses detected!


#### Summary 

This lambda is a recursive AST walker:

> Checks if the node is valid.

> Recursively visits all children nodes.

When it finds a string literal:

> Checks if it violates the “single quotes preferred” rule.

> Records it in offenses.

### Let’s get a better understanding of the AST!

In [11]:
`ruby-parse -v`

"ruby-parse based on parser version 3.3.10.0\n"

---
## Lets you see how Ruby interprets code internally.
### AST (Abstract Syntax Tree) - RoboCop Parser

Useful for writing linters, analyzers, or code transformations, like what RuboCop does.

Helps you understand method calls, operators, and literals at the AST level.

In [12]:
result = `ruby-parse -L -e "2+2"`
puts result


s(:send,
  s(:int, 2), :+,
  s(:int, 2))
2+2
 ~ selector   
~~~ expression
s(:int, 2)
2+2
~ expression
s(:int, 2)
2+2
  ~ expression


### Command:
`ruby-parse -L -e "2+2"`

__ruby-parse__ is part of the __parser gem__. It __parses__ Ruby code and __outputs__ its __AST (Abstract Syntax Tree)__.

`-L` enables location information for expressions in the AST.

`-e "2+2"` tells it to parse the code `"2+2"` from the command line.

### Result: 
```
s(:send,
  s(:int, 2), :+,
  s(:int, 2))
```

#### This is the AST (Abstract Syntax Tree) representation of 2+2:

`s(...)` → AST node constructor.

`:send` → method call node (sending the + message to the left-hand side).

`s(:int, 2)` → integer literal node with value 2.

`:+` → the method being called (here, addition).

So this structure says:

`“Call the + method on integer 2 with argument 2.”`


`2+2` → the source code.

`~ selector` → the operator +.

`~~~ expression` → spans the whole node’s expression.

`s(:int, 2)` → integer literals in the AST.

It’s basically a visual mapping between the AST nodes and the original source code.

-----
Perfect — what i’ve built is a __hands-on exploration of Ruby’s Abstract Syntax Tree (AST)__ using both the __parser gem__ and the __ruby-parse CLI__.

The differences between __parser__ and __ruby-parse__ :
👇

#### 🧩 1. parser (the library)

Gem name: parser

`Maintained by`: [whitequark](https://github.com/whitequark) (the same gem used internally by RuboCop)

`Purpose`: It’s a Ruby library that converts Ruby source code into an Abstract Syntax Tree (AST).

`API`: Used programmatically — you use classes like Parser::CurrentRuby, Parser::Source::Buffer, etc.

#### 🧠 2. ruby-parse (the CLI tool)

`Comes with`: the parser gem
(installed automatically when you install parser)

`Purpose`: A command-line interface to the parser gem — it lets you quickly inspect or visualize the AST from the terminal.

Here’s a solid, clear introduction you could use at the top of your notebook or article 👇

----
#### 🔍 Exploring Ruby’s Abstract Syntax Tree (AST)

In this session, we’re peeking under Ruby’s hood — into how source code becomes structure.
By combining the _parser gem_ (used internally by tools like RuboCop) and the _ruby-parse CLI_, we can visualize and analyze the AST that represents our Ruby programs.

Each snippet below takes a small Ruby example and turns it into a tree of nodes that describe the program’s semantics — not how it looks, but what it means.

You’ll see how:

> 1. Parser::CurrentRuby tokenizes and parses Ruby code into an AST in memory.

> 2. ruby-parse performs the same job externally, showing a human-readable tree in your terminal.

> 3. You can traverse and pretty-print that tree, exploring how Ruby represents methods, strings, and interpolations.

> 4. This process is the foundation of static code analysis — how tools like RuboCop “read” your code without running it.

In short, this walkthrough helps you understand how static analyzers see your Ruby code as data — the first step toward writing your own code linter or formatter.

In [13]:
require 'parser/ruby34'

code = 'puts "Hello"'
buffer = Parser::Source::Buffer.new('(example)')
buffer.source = code
parser = Parser::CurrentRuby.new
ast = parser.parse(buffer)
#p ast

s(:send, nil, :puts,
  s(:str, "Hello"))

In [14]:
result = `ruby-parse -e "puts 'Hello'" --emit-ast`
puts result

In [15]:
result = `ruby-parse -e "puts 'Hello'" --legacy`
puts result


(send nil :puts
  (str "Hello"))


In [16]:
code1 = <<~RUBY
  name = "Gilberto"
  puts "Hello, \#{name}"
  puts "Static analysis is cool"
RUBY

"name = \"Gilberto\"\nputs \"Hello, \#{name}\"\nputs \"Static analysis is cool\"\n"

In [17]:
# Use shell escaping to safely pass the code to ruby-parse
require 'shellwords'
result = `ruby-parse -e #{Shellwords.escape(code1)} --legacy`
puts result

(begin
  (lvasgn :name
    (str "Gilberto"))
  (send nil :puts
    (dstr
      (str "Hello, ")
      (begin
        (lvar :name))))
  (send nil :puts
    (str "Static analysis is cool")))


🧠 Why Shellwords.escape matters

It ensures your Ruby code (which may contain spaces, quotes, or newlines) is safely converted into a single argument for the shell.
Without it, the shell would treat each line as a separate command or argument.

In [18]:
require 'parser/current'
require 'pp'

# 1️⃣ Your Ruby code
code1 = <<~RUBY
    def greet(name)
     puts "Hello, \#{name}"
    end
RUBY

# 2️⃣ Create a buffer and parse it into an AST
buffer = Parser::Source::Buffer.new('(example)')
buffer.source = code1
parser = Parser::CurrentRuby.new
ast = parser.parse(buffer)

# 3️⃣ Pretty-print the AST
#pp ast

s(:def, :greet,
  s(:args,
    s(:arg, :name)),
  s(:send, nil, :puts,
    s(:dstr,
      s(:str, "Hello, "),
      s(:begin,
        s(:lvar, :name)))))

In [19]:
require 'ast'
puts ast.inspect

s(:def, :greet,
  s(:args,
    s(:arg, :name)),
  s(:send, nil, :puts,
    s(:dstr,
      s(:str, "Hello, "),
      s(:begin,
        s(:lvar, :name)))))


A simple, clear, hierarchical visualization of the AST inside your Jupyter cell


In [20]:
require 'parser/current'

# Recursive pretty printer for AST
def print_ast(node, indent = 0)
  return unless node.is_a?(Parser::AST::Node)

  # Print node type
  print "#{' ' * indent}- #{node.type}"

  # Show literal or symbol children inline (for readability)
  literals = node.children.reject { |c| c.is_a?(Parser::AST::Node) }
  print " → #{literals.inspect}" unless literals.empty?
  puts

  # Recurse into child nodes
  node.children.each do |child|
    print_ast(child, indent + 2) if child.is_a?(Parser::AST::Node)
  end
end

# --- Example code ---
code = <<~RUBY
  name = "Gilberto"
  puts "Hello, \#{name}"
  puts "Static analysis is cool"
RUBY

buffer = Parser::Source::Buffer.new('(example)')
buffer.source = code
parser = Parser::CurrentRuby.new
ast = parser.parse(buffer)

# --- Print the AST tree ---
#print_ast(ast)


s(:begin,
  s(:lvasgn, :name,
    s(:str, "Gilberto")),
  s(:send, nil, :puts,
    s(:dstr,
      s(:str, "Hello, "),
      s(:begin,
        s(:lvar, :name)))),
  s(:send, nil, :puts,
    s(:str, "Static analysis is cool")))

| Node     | Meaning                               | Example            |
| -------- | ------------------------------------- | ------------------ |
| `sbegin` | sequence of expressions               | top-level script   |
| `lvasgn` | local variable assignment             | `name = ...`       |
| `str`    | simple string                         | `"Gilberto"`       |
| `dstr`   | *dynamic* string (with interpolation) | `"Hello, #{name}"` |
| `send`   | method call                           | `puts ...`         |

Thank You!